In [6]:
import pandas as pd

pth_models = "/home/sondors/Documents/price/ColBERT_data/18_categories/test/models_18_categories.csv"
pth_offers_train = "/home/sondors/Documents/price/BERT_data/data/13_categories/triplets_13_categories_filtered_train.csv"

df_models = pd.read_csv(pth_models, sep=";")
df_offers_train = pd.read_csv(pth_offers_train, sep=";")

clms = [f'model_id_pred_{i+1}' for i in range(5)] + [f'similarity_{i+1}' for i in range(5)] + ['true_match', 'false_match']
df_offers_train = df_offers_train.drop(columns=clms)

df_offers_train

,name,model_id,category_name,category_id
0,Teac R-1 black,293450,"радиоприемники, радиобудильники, радиочасы",3903
1,Tivoli Audio Model Three black/silver (M3BLK),293587,"радиоприемники, радиобудильники, радиочасы",3903
2,Tivoli Audio Model One black/black (M1BLK),293586,"радиоприемники, радиобудильники, радиочасы",3903
3,Tivoli Audio Portable Audio Laboratory electri...,293460,"радиоприемники, радиобудильники, радиочасы",3903
4,Tivoli Audio Model One white/silver (M1WHT),293586,"радиоприемники, радиобудильники, радиочасы",3903
...,...,...,...,...
569170,Планшет Blackview Tab 11 SE Cortex SC9863A (1....,4898118,планшетные компьютеры и мини-планшеты,510401
569171,Планшет Digma CITI Kids 10 MT8321 4C 2/32Gb 10...,4908671,планшетные компьютеры и мини-планшеты,510401
569172,Планшет Xiaomi Redmi Pad (6 / 128 Gb / Зеленый...,6196024,планшетные компьютеры и мини-планшеты,510401
569173,"Планшет HUAWEI MatePad 10.4"" (2022) 4+64GB WiF...",4903255,планшетные компьютеры и мини-планшеты,510401


In [2]:
df_models

,model_id,name,brand_name,full_name,category_name,category_id
0,623742,920-005619,Logitech,Logitech 920-005619,"чехлы, обложки для гаджетов (телефонов, планше...",3994
1,721952,Zipper Bag,Hama,Hama Zipper Bag,"чехлы, обложки для гаджетов (телефонов, планше...",3994
2,721970,CC-3064,Nokia,Nokia CC-3064,"чехлы, обложки для гаджетов (телефонов, планше...",3994
3,751488,CKS-X7/R,Sony,Sony CKS-X7/R,"чехлы, обложки для гаджетов (телефонов, планше...",3994
4,751989,EP-031023,Era Pro,Era Pro EP-031023,"чехлы, обложки для гаджетов (телефонов, планше...",3994
...,...,...,...,...,...,...
99610,7049424,MD-108,Mivo,Mivo MD-108,портативная акустика,3904
99611,7049425,MD-165,Mivo,Mivo MD-165,портативная акустика,3904
99612,7049426,Boost 20W,Rocket,Rocket Boost 20W,портативная акустика,3904
99613,7049427,Motion 10W,Rocket,Rocket Motion 10W,портативная акустика,3904


In [14]:
def df_balancing(df_raw, filepath, fld, thresh_lower=10, thresh_upper=10):
    df = df_raw[df_raw['model_id']!=0]
    df = df.drop_duplicates(subset='name', keep="last")
    df = df.groupby('model_id').filter(lambda x: len(x) >= thresh_lower)

    ans = [y for x, y in df.groupby('model_id')]
    for i in range(len(ans)):
        #ans[i] = ans[i].drop_duplicates(subset='name', keep="last")
        ans[i] = ans[i].sample(frac=1).sample(frac=1)[:thresh_upper]
    if len(ans)==0:
        print('ERROR\n', filepath)
        #print('df_raw\n', df_raw)
        #print('df\n', df)
        df_new = []
        filename = filepath.split('/')[-1].split('.')[0]
        df_raw.to_csv(f'{fld}/{filename}_ERROR.csv', sep=';',index=False)
    elif len(ans)==1:
        df_new = ans[0]
    else:
        df_new = pd.concat(ans, axis=0).reset_index(drop=True)
    return df_new

fld = "/home/sondors/Documents/price/ColBERT/DATASET"
df_offers_train_balanced = df_balancing(df_offers_train, 'filepath', fld, thresh_lower=0, thresh_upper=5)
df_offers_train_balanced = df_offers_train_balanced.rename(columns={'name': 'full_name'})

df_offers_train_balanced

,full_name,model_id,category_name,category_id
0,Nokia 8800 Sirocco Edition (Цвет:Серебристый),66987,мобильные телефоны,2801
1,Nokia 8800 Sirocco edition Black,66987,мобильные телефоны,2801
2,Nokia 8800 Sirocco Edition (Цвет:Черный),66987,мобильные телефоны,2801
3,Nokia 8800 Sirocco edition Silver,66987,мобильные телефоны,2801
4,Nokia 8800 Sirocco edition Gold,66987,мобильные телефоны,2801
...,...,...,...,...
92196,Xiaomi Redmi Note 11SE (8 / 128 Gb / Синий (Bl...,6476930,мобильные телефоны,2801
92197,Смартфон Xiaomi RedMi Note 11SE CN 5G 8/128 Гб...,6476930,мобильные телефоны,2801
92198,"Смартфон Xiaomi Redmi Note 11SE 8/128 ГБ CN, с...",6476930,мобильные телефоны,2801
92199,Смартфон Xiaomi Redmi Note 11SE 8/128 Shadow B...,6476930,мобильные телефоны,2801


In [13]:
df_offers_train_balanced.model_id.value_counts()

66987      5
4298690    5
4300021    5
4300020    5
4299812    5
          ..
705107     1
705111     1
705121     1
705128     1
3025576    1
Name: model_id, Length: 24721, dtype: int64

In [15]:
models_and_offers = pd.concat([df_models, df_offers_train_balanced], ignore_index=True)
models_and_offers

,model_id,name,brand_name,full_name,category_name,category_id
0,623742,920-005619,Logitech,Logitech 920-005619,"чехлы, обложки для гаджетов (телефонов, планше...",3994
1,721952,Zipper Bag,Hama,Hama Zipper Bag,"чехлы, обложки для гаджетов (телефонов, планше...",3994
2,721970,CC-3064,Nokia,Nokia CC-3064,"чехлы, обложки для гаджетов (телефонов, планше...",3994
3,751488,CKS-X7/R,Sony,Sony CKS-X7/R,"чехлы, обложки для гаджетов (телефонов, планше...",3994
4,751989,EP-031023,Era Pro,Era Pro EP-031023,"чехлы, обложки для гаджетов (телефонов, планше...",3994
...,...,...,...,...,...,...
191811,6476930,NaN,NaN,Xiaomi Redmi Note 11SE (8 / 128 Gb / Синий (Bl...,мобильные телефоны,2801
191812,6476930,NaN,NaN,Смартфон Xiaomi RedMi Note 11SE CN 5G 8/128 Гб...,мобильные телефоны,2801
191813,6476930,NaN,NaN,"Смартфон Xiaomi Redmi Note 11SE 8/128 ГБ CN, с...",мобильные телефоны,2801
191814,6476930,NaN,NaN,Смартфон Xiaomi Redmi Note 11SE 8/128 Shadow B...,мобильные телефоны,2801


In [19]:
models_and_offers.to_csv("/home/sondors/Documents/price/ColBERT_data/18_categories/test/offers_and_models_18_categories.csv", sep=';',index=False)
